In [ ]:
import torch

In [ ]:
print(torch.__version__)

1.13.1+cu116


In [ ]:
!pip install torchdata==0.5.1 -q
!pip install sacremoses -q
!pip install transformers -q
!pip install datasets -q
!pip install pytorch_metric_learning -q
!pip install sentence_transformers -q
!pip install numpy requests nlpaug -q
!pip install pytorch-lightning -q
!pip install nlpaug -q

In [ ]:
import numpy as np
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import itertools
import pandas as pd
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from transformers.utils import logging
logging.set_verbosity(40) 
logging.enable_progress_bar()

In [ ]:
from torchtext import datasets
from torch.utils.data import DataLoader
from transformers import EarlyStoppingCallback

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NLP_Projekt/
from RobertaContrastiveModel import *

Mounted at /content/drive
/content/drive/MyDrive/MojePliki/Studia aktualne/NLP_Projekt


# Data loading tests

In [ ]:
from torchtext._internal.module_utils import is_module_available
is_module_available("torchdata")

True

In [ ]:
yelp_train = datasets.AmazonReviewPolarity(split = 'train')
yelp_test = datasets.AmazonReviewPolarity(split = 'test')

In [ ]:
train_dataloader = DataLoader(yelp_train, batch_size=100, shuffle=True)

In [ ]:
labels, sentences = next(iter(train_dataloader))

In [ ]:
df = pd.DataFrame({'sentence':sentences, 'label':labels})
df['label'][df.label == 1] = 0
df['label'][df.label == 2] = 1

In [ ]:
df.sentence[2]

"Did not fit 2004 Kia Optima LX I purchased this for my 2004 Kia Optima LX. When I went to install it, everything seemed good. But the car was about an inch wider than the hitch and the hitch was not centered in the car. So either the hitch was off center or the car's mounts were. Otherwise seemed like a great hitch."

In [ ]:
df.label[2]

0

# Data augmentation tests

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[2000:2100,:]
df = df.rename(columns={'review': 'sentence', 'sentiment': 'label'})
df['label'][df.label == 'positive'] = 1
df['label'][df.label == 'negative'] = 0

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
# stopwords to be ignored in augmentation
STOPWORDS = ['br','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'one', 
             'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
             'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
             'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
             'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thats', 'would',
             "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
             'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 
             'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 
             'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
             'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
             'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
             'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
             's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll',
             'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
             'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
             'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
# aug = naw.SynonymAug(aug_src='wordnet',aug_min=int(len(df.sentence[0])/3), stopwords=STOPWORDS)
# aug = naw.BackTranslationAug(
#     from_model_name='facebook/wmt19-en-de', 
#     to_model_name='facebook/wmt19-de-en',
#     device='cuda',
#     max_length=64
# )
aug = nas.AbstSummAug(
    model_path='t5-base',
    device='cuda',
    max_length = 64,
    min_length = 20,
)
# aug = naw.ContextualWordEmbsAug(
#     model_path='bert-base-uncased', action="substitute", device='cuda',aug_min=int(len(df.sentence[0])/3), stopwords=STOPWORDS)
# augmented_text = aug.augment(text)

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
%%time
augmented = aug.augment(list(df.sentence)[0])

CPU times: user 2.27 s, sys: 19 ms, total: 2.29 s
Wall time: 2.3 s


In [ ]:
augmented[0]

"'Stranded in Space' originally aired on tv in 1972. in 1991 it was re-released on the mst3k - a never to be made series. the alien world is run as an orwellian 'perfect' society. where, for totally inexplicable reasons, everyone speaks english and drives late model american cars."

In [ ]:
# augmentation methods tests

In [ ]:
%%time
new_sentences = []
new_labels = []
for i in range(len(df)):
    augmented = aug.augment(df.sentence[i])
    for elem in augmented:
        new_sentences.append(elem)
        new_labels.append(df.label[i])

CPU times: user 2.81 s, sys: 3.01 ms, total: 2.81 s
Wall time: 2.81 s


# Utils

In [ ]:
def augment_dataset(df, method='none', n=1, p=1/3):
    STOPWORDS = ['br','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'one', 
             'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 
             'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
             'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
             'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thats', 'would',
             "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
             'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 
             'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 
             'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
             'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
             'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
             'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
             's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll',
             'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
             'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
             'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if method=='none':
        return df
    elif method=='synonym':
        aug = naw.SynonymAug(aug_src='wordnet',aug_min=int(len(df.sentence[0])*p), stopwords=STOPWORDS)
        new_sentences = []
        new_labels = []
        for i in range(len(df)):
            augmented = aug.augment(df.sentence[i], n=n)
            for elem in augmented:
                new_sentences.append(elem)
                new_labels.append(df.label[i])
        new_sentences = new_sentences + list(df.sentence)
        new_labels = new_labels + list(df.label)
        new_df = pd.DataFrame({'sentence':new_sentences, 'label':new_labels})
        return new_df
    elif method=='context':
        aug = naw.ContextualWordEmbsAug(
            model_path='bert-base-uncased', action="substitute", device='cuda',
                aug_min=int(len(df.sentence[0])*p), stopwords=STOPWORDS, batch_size = 64)
        new_sentences = aug.augment(list(df.sentence))
        new_labels = list(df.label) + list(df.label)
        new_sentences = new_sentences + list(df.sentence)
        new_df = pd.DataFrame({'sentence':new_sentences, 'label':new_labels})
        return new_df
    elif method=='translation':
        aug = naw.BackTranslationAug(
            from_model_name='facebook/wmt19-en-de', 
            to_model_name='facebook/wmt19-de-en',
            device='cuda',
            batch_size = 20,
            max_length=64
        )
        new_sentences = aug.augment(list(df.sentence))
        new_labels = list(df.label) + list(df.label)
        new_sentences = new_sentences + list(df.sentence)
        new_df = pd.DataFrame({'sentence':new_sentences, 'label':new_labels})
        return new_df
    elif method == 'summarise':
        aug = nas.AbstSummAug(
            model_path='t5-small',
            device='cuda',
            max_length = 64,
            min_length = 20
        )
        new_sentences = []
        new_labels = []
        for i in range(len(df)):
            augmented = aug.augment(df.sentence[i], n=n)
            for elem in augmented:
                new_sentences.append(elem)
                new_labels.append(df.label[i])
        new_sentences = new_sentences + list(df.sentence)
        new_labels = new_labels + list(df.label)
        new_df = pd.DataFrame({'sentence':new_sentences, 'label':new_labels})
        return new_df
    else:
        raise ValueError(f'Unknown augmentation method: {method}.')

In [ ]:
def prepare_dataset(dataset="yelp", test=False, seed=42,):
    if dataset == 'imdb':
        df = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[:1000,:]
        if test:
            df = pd.read_csv("/content/drive/MyDrive/NLP_Projekt/IMDB Dataset.csv").iloc[1000:10000,:]
        df = df.rename(columns={'review': 'sentence', 'sentiment': 'label'})
        df['label'][df.label == 'positive'] = 1
        df['label'][df.label == 'negative'] = 0
    elif dataset == 'yelp':
        if test:
            size = 9000
            data = datasets.YelpReviewPolarity(split = 'test')
        else:
            size = 1000
            data = datasets.YelpReviewPolarity(split = 'train')
        train_dataloader = DataLoader(data, batch_size=size, shuffle=True)
        labels, sentences = next(iter(train_dataloader))
        df = pd.DataFrame({'sentence':sentences, 'label':labels})
        df['label'][df.label == 1] = 0
        df['label'][df.label == 2] = 1
    elif dataset == 'amazon':
        if test:
            size = 9000
            data = datasets.AmazonReviewPolarity(split = 'test')
        else:
            size = 1000
            data = datasets.AmazonReviewPolarity(split = 'train')
        train_dataloader = DataLoader(data, batch_size=size, shuffle=True)
        labels, sentences = next(iter(train_dataloader))
        df = pd.DataFrame({'sentence':sentences, 'label':labels})
        df['label'][df.label == 1] = 0
        df['label'][df.label == 2] = 1
    else:
        raise ValueError(f'Cannot load the dataset: {dataset}.')
    df = shuffle(df, random_state=seed)
    return df

In [ ]:
class CLDatasetClassification(torch.utils.data.Dataset):
    def __init__(self, index, df, tokenizer, max_length, sample_size=-1):
        if len(index) == 0:
            texts = df["sentence"].tolist()
            labels = df["label"].tolist()
        else:
            if sample_size != -1:
                index = np.random.choice(index, sample_size, replace=False)
            texts = df.iloc[index]["sentence"].tolist()
            labels = df.iloc[index]["label"].tolist()
        train_encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)

        self.encodings = train_encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
def cross_validate(args):
    augmentation = args.augmentation
    n_aug = args.n_aug
    p = args.p_aug
    seed = args.seed
    max_length = args.max_length
    learning_rate = args.learning_rate
    num_warmup_steps = args.num_warmup_steps
    num_training_steps = args.num_training_steps
    eps = args.eps
    model_name = args.model_name
    model_type = args.model_type
    weight_decay = args.weight_decay
    la = args.la
    gamma = args.gamma
    margin = args.margin
    centers = args.centers
    beta = args.beta
    save_steps = args.save_steps
    sample_size = args.sample_size
    n_split = args.n_split
    supcon_temp = args.supcon_temp
    np.random.seed(seed)
    dataset_name = args.dataset_name
    softmax_scale = args.softmax_scale
    alpha = args.alpha
    extended_inference = args.extended_inference

    training_args = TrainingArguments(
        output_dir=args.output_dir,
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.per_device_train_batch_size,
        per_device_eval_batch_size=args.per_device_eval_batch_size,
        save_steps=save_steps,
        # evaluation_strategy='steps',
        # eval_steps=8,
        # eval_delay=40,
        # load_best_model_at_end = True,
        # metric_for_best_model='eval_loss',
        # greater_is_better = False
    )

    kf = KFold(n_splits=n_split, random_state=seed, shuffle=True)
    cross_val_res = {}
    df = prepare_dataset(dataset_name)
    if model_name == "roberta-base":
        embedding_size = 768
    else:
        embedding_size = 1024

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    for fold_id, (train_index, valid_index) in enumerate(kf.split(df)):
        # clear cache
        torch.cuda.empty_cache()
        trainer = []
        
        if augmentation != 'none':
            train_index = np.random.choice(train_index, sample_size, replace=False)
            df_train = df.iloc[train_index].reset_index()
            df_train = augment_dataset(df_train, method=augmentation, n=n_aug, p=p)
            train_dataset = CLDatasetClassification([], df_train, tokenizer, max_length)
        else:
            train_dataset = CLDatasetClassification(train_index, df, tokenizer, max_length, sample_size)

        torch.cuda.empty_cache()
        
        valid_dataset = CLDatasetClassification(valid_index, df, tokenizer, max_length)
        if model_type == "softriple":
            clf_loss = losses_ml.SoftTripleLoss(num_classes=len(unique(df.label)), embedding_size=embedding_size,
                                             centers_per_class=centers, la=la, gamma=gamma, margin=margin)
        elif model_type == "supcon":
            clf_loss = losses_ml.SupConLoss(temperature=supcon_temp)
        elif model_type == "proxynca":
            clf_loss = losses_ml.ProxyNCALoss(len(unique(df.label)), embedding_size=embedding_size,
                                           softmax_scale=softmax_scale)
        elif model_type == "proxyanchor":
            clf_loss = losses_ml.ProxyAnchorLoss(len(unique(df.label)), embedding_size=embedding_size, margin=margin,
                                              alpha=alpha)
        elif model_type == "npairs":
            clf_loss = losses_ml.NPairsLoss()
        elif model_type == "triplet":
            clf_loss = BatchAllTripletLoss()
        elif model_type == "baseline":
            clf_loss = None
        else:
            raise ValueError(
                f'The model_type: {model_type} is not supported. Choose one of following: triple_entropy, supcon, baseline.')
        model = RobertaContrastiveLearning.from_pretrained(model_name,
                                                           num_labels=len(
                                                               unique(df.label)),
                                                           clf_loss=clf_loss,
                                                           beta=beta,
                                                           extended_inference=extended_inference)
        param_groups = [{"params": model.parameters(),
                         'lr': float(learning_rate)}]
        # early_stop_callback = EarlyStoppingCallback(early_stopping_patience=1)

        optimizer = transformers.AdamW(param_groups, eps=eps, weight_decay=weight_decay, correct_bias=True)
        scheduler = transformers.get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                                 num_warmup_steps=num_warmup_steps,
                                                                 num_training_steps=num_training_steps)
        optimizers = optimizer, scheduler
        set_seed(seed)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            optimizers=optimizers,
            # callbacks=[early_stop_callback],
            compute_metrics=compute_metrics
        )

        trainer.train()

        test_df = prepare_dataset(dataset_name, True)
        test_data = CLDatasetClassification(np.arange(len(test_df)), test_df, tokenizer, 64)
        cross_val_res[fold_id] = trainer.evaluate(eval_dataset=test_data)
        #trainer.save_model(args.output_dir + f'{fold_id}_{model_name}_{model_type}_{dataset_name}/model')
    results = []
    for measure in ["eval_f1_score", "eval_recall_score", "eval_accuracy_score", "eval_precision_score"]:
        m = np.mean([el[measure] for el in cross_val_res.values()])
        std = np.std([el[measure] for el in cross_val_res.values()])
        results.append([m,std])
    trainer.save_model(args.output_dir + f'/{augmentation}_{model_name}_{model_type}_{dataset_name}/model')
    return results, trainer

# Model hyperparameters

In [ ]:
def create_args(n, use_rlarge, augmentation='none', n_aug=1, p_aug=1/3, n_split=5, dataset='yelp'):
    if use_rlarge:
        modeln = "roberta-large"
    else:
        modeln = "roberta-base"

    args = Namespace(
        augmentation = augmentation,
        n_aug = n_aug,
        p_aug = p_aug,
        max_length = 64,
        learning_rate=1e-5,
        num_warmup_steps=10,
        eps=1e-08,
        model_name=modeln,
        model_type="softriple",
        weight_decay=0.01,
        la=8,
        supcon_temp=0.1,
        gamma=0.1,
        margin=0.1,
        centers=5,
        beta=0.4,
        seed=2048,
        output_dir="./result",
        save_steps=2000,
        epochs=100,
        num_training_steps=100,
        per_device_train_batch_size=50,
        per_device_eval_batch_size=50,
        sample_size=n,
        n_split=n_split,
        dataset_name=dataset,
        softmax_scale=1,
        alpha=0.1,
        extended_inference=0.1)
    return args

# Base model training - yelp

In [ ]:
args = create_args(100, False, augmentation='none', dataset='yelp')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

Step,Training Loss


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/none_roberta-base_softriple_yelp/model
Configuration saved in ./result/none_roberta-base_softriple_yelp/model/config.json
Model weights saved in ./result/none_roberta-base_softriple_yelp/model/pytorch_model.bin


CPU times: user 12min 20s, sys: 10.8 s, total: 12min 31s
Wall time: 12min 23s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8445555555555556 std:  0.010728064038990988
valid f1:  0.8443757274935448 std:  0.010628918087203855


# Base model training - amazon

In [ ]:
args = create_args(100, False, augmentation='none', dataset='amazon')

In [ ]:
%%time
results, trainer = cross_validate(args)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

{'train_runtime': 107.2897, 'train_samples_per_second': 93.206, 'train_steps_per_second': 1.864, 'train_loss': 0.17241645812988282, 'epoch': 100.0}


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50
Saving model checkpoint to ./result/none_roberta-base_softriple_amazon/model


{'eval_loss': 0.29758763313293457, 'eval_accuracy_score': 0.8471111111111111, 'eval_f1_score': 0.8469766673011535, 'eval_recall_score': 0.8074170461938842, 'eval_precision_score': 0.8841130372833056, 'eval_runtime': 31.7841, 'eval_samples_per_second': 283.16, 'eval_steps_per_second': 5.663, 'epoch': 100.0}


Configuration saved in ./result/none_roberta-base_softriple_amazon/model/config.json
Model weights saved in ./result/none_roberta-base_softriple_amazon/model/pytorch_model.bin


CPU times: user 12min 24s, sys: 17.3 s, total: 12min 41s
Wall time: 13min 34s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8464666666666666 std:  0.003509000772264536
valid f1:  0.8461544828876768 std:  0.0037097316361721084


# Base model training - imdb

In [ ]:
args = create_args(100, False, augmentation='none', dataset = 'imdb')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/none_roberta-base_softriple_imdb/model
Configuration saved in ./result/none_roberta-base_softriple_imdb/model/config.json
Model weights saved in ./result/none_roberta-base_softriple_imdb/model/pytorch_model.bin


CPU times: user 12min 25s, sys: 11.2 s, total: 12min 37s
Wall time: 12min 32s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7560888888888889 std:  0.030502896037371025
valid f1:  0.7527231736398037 std:  0.03527620799083204


# Synonym augmentation on yelp

In [ ]:
args = create_args(100, False, augmentation='synonym', n_aug=1, p_aug=1/4, dataset = 'yelp')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/synonym_roberta-base_softriple_yelp/model
Configuration saved in ./result/synonym_roberta-base_softriple_yelp/model/config.json
Model weights saved in ./result/synonym_roberta-base_softriple_yelp/model/pytorch_model.bin


CPU times: user 21min, sys: 20.5 s, total: 21min 21s
Wall time: 21min 32s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8428222222222221 std:  0.013812250779830861
valid f1:  0.8422330291167783 std:  0.014748464277625259


# Synonym augmentation on amazon

In [ ]:
args = create_args(100, False, augmentation='synonym', n_aug=1, p_aug=1/4, dataset = 'amazon')

In [ ]:
%%time
results, trainer = cross_validate(args)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


...

{'train_runtime': 228.2466, 'train_samples_per_second': 87.625, 'train_steps_per_second': 1.752, 'train_loss': 0.13376535415649415, 'epoch': 100.0}


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50
Saving model checkpoint to ./result/synonym_roberta-base_softriple_amazon/model
Configuration saved in ./result/synonym_roberta-base_softriple_amazon/model/config.json


{'eval_loss': 0.3305967450141907, 'eval_accuracy_score': 0.8761111111111111, 'eval_f1_score': 0.876128510866648, 'eval_recall_score': 0.8654513888888888, 'eval_precision_score': 0.8895828686147669, 'eval_runtime': 34.7026, 'eval_samples_per_second': 259.347, 'eval_steps_per_second': 5.187, 'epoch': 100.0}


Model weights saved in ./result/synonym_roberta-base_softriple_amazon/model/pytorch_model.bin


CPU times: user 22min 36s, sys: 25.8 s, total: 23min 2s
Wall time: 23min 26s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8341111111111111 std:  0.023173526828668342
valid f1:  0.8325954895740741 std:  0.024386822574846964


# Synonym augmentation on imdb

In [ ]:
args = create_args(100, False, augmentation='synonym', n_aug=1, p_aug=1/4, dataset = 'imdb')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/synonym_roberta-base_softriple_imdb/model
Configuration saved in ./result/synonym_roberta-base_softriple_imdb/model/config.json
Model weights saved in ./result/synonym_roberta-base_softriple_imdb/model/pytorch_model.bin


CPU times: user 22min 45s, sys: 20.3 s, total: 23min 5s
Wall time: 22min 51s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7486444444444444 std:  0.006298500703430208
valid f1:  0.7475392093087847 std:  0.006127585838102354


# Context embedding augmentation on yelp

In [ ]:
args = create_args(100, False, augmentation='context', p_aug=1/4, dataset = 'yelp')

In [ ]:
%%capture
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8069555555555554 std:  0.027775342115438122
valid f1:  0.8034795111653128 std:  0.03275316100658348


In [ ]:
del trainer

# Context embedding augmentation on amazon

In [ ]:
args = create_args(100, False, augmentation='context', p_aug=1/4, dataset = 'amazon')

In [ ]:
%%capture
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8426666666666666 std:  0.029197074096602015
valid f1:  0.8404842200250198 std:  0.030788990099952295


In [ ]:
del trainer

# Context embedding augmentation on imdb

In [ ]:
args = create_args(100, False, augmentation='context', p_aug=1/4, dataset = 'imdb')

In [ ]:
%%capture
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7713555555555555 std:  0.0037910876643797607
valid f1:  0.7690457341105259 std:  0.0043922534097935615


In [ ]:
del trainer

# Back-translation augmentation on yelp

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='translation', dataset = 'yelp')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/translation_roberta-base_softriple_yelp/model
Configuration saved in ./result/translation_roberta-base_softriple_yelp/model/config.json
Model weights saved in ./result/translation_roberta-base_softriple_yelp/model/pytorch_model.bin


CPU times: user 25min 57s, sys: 22.1 s, total: 26min 19s
Wall time: 26min 35s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8027333333333333 std:  0.029497989050521582
valid f1:  0.8012682726183534 std:  0.030942658980085525


# Back-translation augmentation on amazon

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='translation', dataset = 'amazon')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/translation_roberta-base_softriple_amazon/model
Configuration saved in ./result/translation_roberta-base_softriple_amazon/model/config.json
Model weights saved in ./result/translation_roberta-base_softriple_amazon/model/pytorch_model.bin


CPU times: user 24min 8s, sys: 22.7 s, total: 24min 31s
Wall time: 24min 53s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8348222222222222 std:  0.023830015309051492
valid f1:  0.8333356464901183 std:  0.025636756885273244


# Back-translation augmentation on imdb

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='translation', dataset = 'imdb')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/translation_roberta-base_softriple_imdb/model
Configuration saved in ./result/translation_roberta-base_softriple_imdb/model/config.json
Model weights saved in ./result/translation_roberta-base_softriple_imdb/model/pytorch_model.bin


CPU times: user 27min 19s, sys: 23.8 s, total: 27min 43s
Wall time: 27min 32s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7367111111111111 std:  0.015877089635970523
valid f1:  0.7329026116503924 std:  0.018069901671520852


# Coping data augmentation on yelp

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='synonym', dataset = 'yelp', n_aug=1, p_aug=0)

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/synonym_roberta-base_softriple_yelp/model
Configuration saved in ./result/synonym_roberta-base_softriple_yelp/model/config.json
Model weights saved in ./result/synonym_roberta-base_softriple_yelp/model/pytorch_model.bin


CPU times: user 22min 23s, sys: 19.5 s, total: 22min 43s
Wall time: 22min 48s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8177555555555557 std:  0.012487702592926482
valid f1:  0.8167820740456978 std:  0.012581263164196946


# Coping data augmentation on amazon

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='synonym', dataset = 'amazon', n_aug=1, p_aug=0)

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/synonym_roberta-base_softriple_amazon/model
Configuration saved in ./result/synonym_roberta-base_softriple_amazon/model/config.json
Model weights saved in ./result/synonym_roberta-base_softriple_amazon/model/pytorch_model.bin


CPU times: user 22min 23s, sys: 20.2 s, total: 22min 43s
Wall time: 22min 49s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8599333333333334 std:  0.015451493195652921
valid f1:  0.8597908967730612 std:  0.015500538441156423


# Coping data augmentation on imdb

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='synonym', dataset = 'imdb', n_aug=1, p_aug=0)

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/pytorch_model.bin
Some weights of the model

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/synonym_roberta-base_softriple_imdb/model
Configuration saved in ./result/synonym_roberta-base_softriple_imdb/model/config.json
Model weights saved in ./result/synonym_roberta-base_softriple_imdb/model/pytorch_model.bin


CPU times: user 22min 54s, sys: 19.6 s, total: 23min 14s
Wall time: 23min 1s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7690444444444444 std:  0.006836214388131296
valid f1:  0.7673109451553471 std:  0.008337168508344083


# Summarization data augmentation on yelp

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='summarise', dataset = 'yelp')

In [ ]:
%%time
results, trainer = cross_validate(args)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Accumulation steps = 1
  Total optimization steps = 400
  Number of trainable parameters = 124654850


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 215.629, 'train_samples_per_second': 92.752, 'train_steps_per_second': 1.855, 'train_loss': 0.14496091842651368, 'epoch': 100.0}


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


{'eval_loss': 0.3988732397556305, 'eval_accuracy_score': 0.8258888888888889, 'eval_f1_score': 0.8259101886731796, 'eval_recall_score': 0.8605623983267487, 'eval_precision_score': 0.7929336188436831, 'eval_runtime': 33.4019, 'eval_samples_per_second': 269.446, 'eval_steps_per_second': 5.389, 'epoch': 100.0}


...

{'eval_loss': 0.369619220495224, 'eval_accuracy_score': 0.8555555555555555, 'eval_f1_score': 0.8556153130409963, 'eval_recall_score': 0.8633511503602138, 'eval_precision_score': 0.8391687372938784, 'eval_runtime': 32.5241, 'eval_samples_per_second': 276.718, 'eval_steps_per_second': 5.534, 'epoch': 100.0}


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Ac

{'train_runtime': 218.6395, 'train_samples_per_second': 91.475, 'train_steps_per_second': 1.829, 'train_loss': 0.12121415138244629, 'epoch': 100.0}


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50
Saving model checkpoint to ./result/summarise_roberta-base_softriple_yelp/model
Configuration saved in ./result/summarise_roberta-base_softriple_yelp/model/config.json


{'eval_loss': 0.40418189764022827, 'eval_accuracy_score': 0.8441111111111111, 'eval_f1_score': 0.8440325302991211, 'eval_recall_score': 0.8270973739251685, 'eval_precision_score': 0.843764817449028, 'eval_runtime': 33.3627, 'eval_samples_per_second': 269.762, 'eval_steps_per_second': 5.395, 'epoch': 100.0}
CPU times: user 25min 25s, sys: 19.6 s, total: 25min 45s
Wall time: 25min 46s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8420444444444444 std:  0.009524134385993767
valid f1:  0.8420334073536475 std:  0.009532253088008496


In [ ]:
del trainer

# Summarization data augmentation on amazon

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='summarise', dataset = 'amazon')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Ac

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Ac

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/summarise_roberta-base_softriple_amazon/model
Configuration saved in ./result/summarise_roberta-base_softriple_amazon/model/config.json


CPU times: user 25min 30s, sys: 25.4 s, total: 25min 56s
Wall time: 26min 12s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.8494444444444443 std:  0.01999888885802298
valid f1:  0.8481958853910211 std:  0.021506099365947128


In [ ]:
del trainer

# Summarization data augmentation on imdb

In [ ]:
torch.cuda.empty_cache()

In [ ]:
args = create_args(100, False, augmentation='summarise', dataset = 'imdb')

In [ ]:
%%time
results, trainer = cross_validate(args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Ac

Step,Training Loss


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

***** Running training *****
  Num examples = 200
  Num Epochs = 100
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 50
  Gradient Ac

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9000
  Batch size = 50


Saving model checkpoint to ./result/summarise_roberta-base_softriple_imdb/model
Configuration saved in ./result/summarise_roberta-base_softriple_imdb/model/config.json


CPU times: user 27min 6s, sys: 20.6 s, total: 27min 27s
Wall time: 27min 27s


In [ ]:
print('valid acc: ', results[2][0], 'std: ', results[2][1])
print('valid f1: ', results[0][0], 'std: ', results[0][1])

valid acc:  0.7680222222222222 std:  0.004650448007092092
valid f1:  0.7659406106854296 std:  0.005099369721020711


In [ ]:
del trainer

# Plot model performance based on augmentation strength

In [ ]:
%%time
accs = []
f1s = []
ps = np.linspace(0,1,11)
counter = 0
for p in ps:
    args = create_args(100, False, augmentation='synonym', n_aug=1, p_aug=p, n_split=2)
    results, trainer = cross_validate(args)
    accs.append(results[2][0])
    f1s.append(results[0][0])
    del trainer
    print('----------------------------------------------------ITERATION {} ENDED----------------------------------------------------'.format(counter))
    counter += 1

In [ ]:
plt.plot(ps,accs)
plt.title('Accuracy for RoBERTa Base model')
plt.xlabel('ratio of augmented words')
plt.ylabel('acc')
plt.show()

In [ ]:
plt.plot(ps,f1s)
plt.title('F1-Score for RoBERTa Base model')
plt.xlabel('ratio of augmented words')
plt.ylabel('F1')
plt.show()